In [52]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
import glob
import h5py
import idx2numpy as idx
import warnings
import re
from PIL import Image
from tqdm import tqdm

# from data_generator.object_detection_2d_data_generator import DataGenerator
# import xml.etree.ElementTree as ET

In [55]:
warnings.filterwarnings('ignore')

#входное разрешение модели
IMAGE_HEIGHT = 28 # Height of the input images
IMAGE_WIDTH = 28 #Width of the input images
WORK_DIR = 'D:\work\\test_comp_vision\datasets\!_lines_w25_parsed_full' #путь к папке с файлами
DATASET_H5 = 'D:\work\\test_comp_vision\datasets\!_lines_w25_dataset.h5'
DATASET_IDX_IMG = r'D:\work\test_comp_vision\datasets\!_lines_w25_dataset_images.idx'
DATASET_IDX_CLS = r'D:\work\test_comp_vision\datasets\!_lines_w25_dataset_classes.idx'
LABELS = '0123456789АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'
EXPORT_NAME_CSV = '!img_classes.csv'

In [6]:
"""
Структура папок:
base_folder
    image_0
        image_0_0.jpg
        image_0_0.txt
        image_0_1.jpg
        image_0_1.txt
        ...
    image_1
        image_1_0.jpg
        image_1_0.txt
        image_1_1.jpg
        image_1_1.txt
        ...
        
"""
# TODO заменить работу с индексами над путями к файлам на нормальную обработку разрешения файлов
def get_dataframe(path):
    # pattern = r'/^(.+)(\.[^ .]+)?$/'      # для удаления разрешения из пути к файлу
    
    symbol_classes = []
    for folder in os.listdir(path):

        for img_file in glob.glob(f'{path}/{folder}/*.jpg'):
            txt_file =  f"{img_file[:-3]}txt"    # меняем разрешение для открытия текста [:-3]
            # print(f'IMG_file: {img_file}\nTXT_file: {txt_file}')
            with open(txt_file, 'r') as f:
                symbol_class = f.read()
                
            value = (img_file.replace('\\', '/'), symbol_class)
            symbol_classes.append(value)
    column_names = ['file_name', 'class']
    return pd.DataFrame(symbol_classes, columns=column_names)

In [7]:
#Создаем DataFrame в котором хранится путь к фалу/изображению и его класс
base_path = WORK_DIR #путь к основной папке
df_text = pd.DataFrame()
df_text = df_text.append(get_dataframe(base_path))
df_text

,file_name,class
0,D:/work/test_comp_vision/datasets/!_lines_w25_...,в
1,D:/work/test_comp_vision/datasets/!_lines_w25_...,е
2,D:/work/test_comp_vision/datasets/!_lines_w25_...,е
3,D:/work/test_comp_vision/datasets/!_lines_w25_...,р
4,D:/work/test_comp_vision/datasets/!_lines_w25_...,а
...,...,...
409215,D:/work/test_comp_vision/datasets/!_lines_w25_...,е
409216,D:/work/test_comp_vision/datasets/!_lines_w25_...,з
409217,D:/work/test_comp_vision/datasets/!_lines_w25_...,а
409218,D:/work/test_comp_vision/datasets/!_lines_w25_...,в


In [38]:
# Преобразуем классы к числовым значениям и меняем формат в ссылках на unix-like (иначе PIL не может открыть файл)
labels_to_int = {}
for i, label in enumerate(LABELS):
    labels_to_int[label] = i
print(labels_to_int)



df_images = pd.DataFrame({'image_path':df_text['file_name'],
                   'class_id':[labels_to_int[label] for label in df_text['class']]})
df_images.to_csv(EXPORT_NAME_CSV, index=False)
df_images

# df = pd.DataFrame({'image_name':df_text['file_name'],
#                   'xmin':0, 'xmax':IMAGE_WIDTH, 'ymin':0, 'ymax':IMAGE_HEIGHT,
#                   'class_id':[labels_to_int[label] for label in df_text['class']]})

#df.to_csv(EXPORT_NAME_CSV, index=False)
#df

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'А': 10, 'Б': 11, 'В': 12, 'Г': 13, 'Д': 14, 'Е': 15, 'Ё': 16, 'Ж': 17, 'З': 18, 'И': 19, 'Й': 20, 'К': 21, 'Л': 22, 'М': 23, 'Н': 24, 'О': 25, 'П': 26, 'Р': 27, 'С': 28, 'Т': 29, 'У': 30, 'Ф': 31, 'Х': 32, 'Ц': 33, 'Ч': 34, 'Ш': 35, 'Щ': 36, 'Ъ': 37, 'Ы': 38, 'Ь': 39, 'Э': 40, 'Ю': 41, 'Я': 42, 'а': 43, 'б': 44, 'в': 45, 'г': 46, 'д': 47, 'е': 48, 'ё': 49, 'ж': 50, 'з': 51, 'и': 52, 'й': 53, 'к': 54, 'л': 55, 'м': 56, 'н': 57, 'о': 58, 'п': 59, 'р': 60, 'с': 61, 'т': 62, 'у': 63, 'ф': 64, 'х': 65, 'ц': 66, 'ч': 67, 'ш': 68, 'щ': 69, 'ъ': 70, 'ы': 71, 'ь': 72, 'э': 73, 'ю': 74, 'я': 75}


,image_path,class_id
0,D:/work/test_comp_vision/datasets/!_lines_w25_...,45
1,D:/work/test_comp_vision/datasets/!_lines_w25_...,48
2,D:/work/test_comp_vision/datasets/!_lines_w25_...,48
3,D:/work/test_comp_vision/datasets/!_lines_w25_...,60
4,D:/work/test_comp_vision/datasets/!_lines_w25_...,43
...,...,...
409215,D:/work/test_comp_vision/datasets/!_lines_w25_...,48
409216,D:/work/test_comp_vision/datasets/!_lines_w25_...,51
409217,D:/work/test_comp_vision/datasets/!_lines_w25_...,43
409218,D:/work/test_comp_vision/datasets/!_lines_w25_...,45


## .H5 файл генератор

In [6]:
#Создаем файл датасета для обучения детектора
train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
images_dir = ''

train_labels_filename = EXPORT_NAME_CSV

train_dataset.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        #input_format=['image_name', 'class_id'],
                        include_classes='all')

train_dataset.create_hdf5_dataset(file_path=DATASET_H5,
                                  resize=[IMAGE_HEIGHT, IMAGE_WIDTH],
                                  variable_image_size=True,
                                  verbose=True)

Creating HDF5 dataset: 100%|██████████████████████████████████████████████████| 409220/409220 [46:10<00:00, 147.69it/s]


## .IDX файл генератор

In [62]:
# преобразуем все изображения в массив np.array()

parsed_images = []              # создаем два файла: в одном хранятся изображения в формате np.array
parsed_classes = []             # во втором хранятся классы

for row in tqdm(df_images[:10].iterrows()):      
    parsed_images.append(np.asarray(Image.open(row[1][0])))  #открываем картинку по ссылке, преобразуем в массив
    #print(parsed_images[row[0]].shape)
    
    parsed_classes.append(row[1][1])
    #print(parsed_classes[row[0]])

np_images = np.asarray(parsed_images) 
np_classes = np.asarray(parsed_classes)

print(np_images.shape)
print(np_classes)

idx.convert_to_file(DATASET_IDX_IMG, np_images)
idx.convert_to_file(DATASET_IDX_CLS, np_classes)

images = idx.convert_from_file(DATASET_IDX_IMG)
lables = idx.convert_from_file(DATASET_IDX_CLS)

print(images.shape)
print(lables)

# idx.convert_to_file(DATASET_IDX, df_images)


10it [00:00, 3343.14it/s]

(10, 28, 28)
[45 48 48 60 43 51 57 43 59 58]
(10, 28, 28)
[45 48 48 60 43 51 57 43 59 58]


In [36]:
#df = pd.read_csv('myfile.csv', sep=',', header=None)
#print(df.values)
im_paths = ["D:/work/test_comp_vision/datasets/!_lines_w25_parsed_full/0_a/0_a_0.jpg",
           "D:/work/test_comp_vision/datasets/!_lines_w25_parsed_full/0_a/0_a_3.jpg"]

emnist_images = "D:\work\\test_comp_vision\datasets\emnist_gzip\emnist-byclass-train-images-idx3-ubyte"
emnist_labels = "D:\work\\test_comp_vision\datasets\emnist_gzip\emnist-byclass-train-labels-idx1-ubyte"
images = idx.convert_from_file(emnist_images)
labels = idx.convert_from_file(emnist_labels)

#print(images.shape)
#print(images[0])

#print(labels.shape)
#print(labels[0])


train_labels_filename = EXPORT_NAME_CSV

parsed_im = []
for im_p in im_paths:
    image = Image.open(im_path)
    parsed_im.append(np.asarray(image))

npdata = np.asarray(parsed_im)    
#npdata = np.asarray(image)


my_dataset = np.array([[[0, 2, 3]], [[0, 2, 3]], [[0, 2, 3]], [[1, 3, 5]]])
print(npdata.shape)


(28, 28)
(28, 28)
(2, 28, 28)


In [18]:
im_path = "D:/work/test_comp_vision/datasets/!_lines_w25_parsed_full/0_a/0_a_0.jpg"

image = cv2.imread(im_path)
print(image.shape)
npdata = np.asarray(image)
print(npdata[:5])

#im_pil = Image.fromarray(image)
#print(im_pil)

image = Image.open(im_path)
print(image.size)
npdata = np.asarray(image)
print(npdata[:5])

#backtorgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
#color_converted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#pil_image=Image.fromarray(color_converted)


(28, 28, 3)
[[[255 255 255]
  [255 255 255]
  [236 236 236]
  [236 236 236]
  [237 237 237]
  [237 237 237]
  [236 236 236]
  [238 238 238]
  [239 239 239]
  [234 234 234]
  [240 240 240]
  [235 235 235]
  [236 236 236]
  [238 238 238]
  [236 236 236]
  [238 238 238]
  [238 238 238]
  [236 236 236]
  [236 236 236]
  [240 240 240]
  [236 236 236]
  [236 236 236]
  [241 241 241]
  [235 235 235]
  [237 237 237]
  [236 236 236]
  [255 255 255]
  [255 255 255]]

 [[254 254 254]
  [254 254 254]
  [240 240 240]
  [237 237 237]
  [238 238 238]
  [237 237 237]
  [237 237 237]
  [239 239 239]
  [236 236 236]
  [236 236 236]
  [238 238 238]
  [238 238 238]
  [238 238 238]
  [237 237 237]
  [237 237 237]
  [237 237 237]
  [236 236 236]
  [239 239 239]
  [238 238 238]
  [238 238 238]
  [236 236 236]
  [237 237 237]
  [235 235 235]
  [239 239 239]
  [237 237 237]
  [236 236 236]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [236 236 236]
  [236 236 236]
  [154 154 154]
  [153 1